In [ ]:
import os
import shutil
from PIL import Image, ImageOps, ImageDraw
from torchvision.transforms import functional as F
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision import models
import random
from torch.optim import lr_scheduler

In [ ]:
def delete_all_in_folder(folder_path):
    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')
    else:
        print(f'The folder {folder_path} does not exist.')
delete_all_in_folder('/kaggle/working/')

In [ ]:
def set_seed(seed_value: int):
    """
    Set the seed for reproducibility in Python, NumPy, and PyTorch.
    Args:
    - seed_value (int): The seed value to use for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_value = 42
set_seed(seed_value)

In [ ]:
class CIFAR100RotationDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.folder = folder
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_to_idx = {}
        class_names = sorted(os.listdir(folder))
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}
        for class_folder in class_names:
            class_path = os.path.join(folder, class_folder)
            if os.path.isdir(class_path):
                for img_name in os.listdir(class_path):
                    if img_name.endswith('.png'):
                        img_path = os.path.join(class_path, img_name)
                        self.image_paths.append(img_path)
                        label = self.class_to_idx[class_folder]
                        self.labels.append(label)
        print(f"Class to index mapping: {self.class_to_idx}")
        print(f"Loaded {len(self.image_paths)} images from {self.folder}")
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        rotation_angle = np.random.choice([0, 90, 180, 270])
        rotated_image = image.rotate(rotation_angle, expand=False)
        if self.transform:
            rotated_image = self.transform(rotated_image)
        label = rotation_angle // 90
        return rotated_image, label

In [ ]:
class ResNetRotationModel(nn.Module):
    def __init__(self):
        super(ResNetRotationModel, self).__init__()
        self.resnet = models.resnet50(pretrained=False)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, 4)
    def forward(self, x):
        return self.resnet(x)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=10):
    model.to(device)
    best_val_acc = 0.0
    best_model_wts = None
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total_samples = 0
        train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} Training", leave=False)
        for inputs, labels in train_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            _, preds = torch.max(outputs, 1)
            batch_corrects = torch.sum(preds == labels.data)
            batch_acc = (batch_corrects.double() / inputs.size(0)).item() * 100
            running_loss += loss.item() * inputs.size(0)
            running_corrects += batch_corrects
            total_samples += inputs.size(0)
            train_bar.set_postfix(
                loss=loss.item(),
                acc=batch_acc
            )
        epoch_loss = running_loss / total_samples
        epoch_acc = (running_corrects.double() / total_samples).item() * 100
        print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f} - Train Acc: {epoch_acc:.2f}%')
        model.eval()
        val_running_loss = 0.0
        val_running_corrects = 0
        val_total_samples = 0
        val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} Validation", leave=False)
        with torch.no_grad():
            for inputs, labels in val_bar:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)
                batch_corrects = torch.sum(preds == labels.data)
                batch_acc = (batch_corrects.double() / inputs.size(0)).item() * 100
                val_running_loss += loss.item() * inputs.size(0)
                val_running_corrects += batch_corrects
                val_total_samples += inputs.size(0)
                val_bar.set_postfix(
                    loss=loss.item(),
                    acc=batch_acc
                )
        val_loss = val_running_loss / val_total_samples
        val_acc = (val_running_corrects.double() / val_total_samples).item() * 100
        print(f'Epoch {epoch+1}/{num_epochs} - Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.2f}%')
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = model.state_dict()
    if best_model_wts:
        model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'best_rotation_model.pth')
def test_model(model, test_loader, criterion, device):
    model.to(device)
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0
    test_bar = tqdm(test_loader, desc="Testing", leave=False)
    with torch.no_grad():
        for inputs, labels in test_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total_samples += inputs.size(0)
            test_bar.set_postfix(
                loss=running_loss / total_samples,
                acc=(running_corrects.double() / total_samples).item() * 100
            )
    epoch_loss = running_loss / total_samples
    epoch_acc = (running_corrects.double() / total_samples).item() * 100
    print(f'Test Loss: {epoch_loss:.4f} - Test Acc: {epoch_acc:.2f}%')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
train_dataset = CIFAR100RotationDataset(folder='/kaggle/input/cifar100/cifar100/train', transform=transform)
test_dataset = CIFAR100RotationDataset(folder='/kaggle/input/cifar100/cifar100/test', transform=transform)
print(f"Number of samples in train dataset: {len(train_dataset)}")
print(f"Number of samples in test dataset: {len(test_dataset)}")
train_idx, val_idx = train_test_split(
    list(range(len(train_dataset))), test_size=0.20, stratify=train_dataset.labels
)
train_set = Subset(train_dataset, train_idx)
val_set = Subset(train_dataset, val_idx)
print(f'Training set size: {len(train_set)}')
print(f'Validation set size: {len(val_set)}')
print(f'Test set size: {len(test_dataset)}')
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)
model = ResNetRotationModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 1e-4)
criterion = nn.CrossEntropyLoss()

In [ ]:
train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=70)

In [ ]:
model.load_state_dict(torch.load('best_rotation_model.pth'))
test_model(model, test_loader, criterion, device)